In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter
from scipy import stats 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN 

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3013572923120,
 'week52high': 183.14,
 'week52low': 117.89,
 'week52highSplitAdjustOnly': 188.5,
 'week52lowSplitAdjustOnly': 119.98,
 'week52change': 0.341213429120285,
 'sharesOutstanding': 16567578034,
 'float': 0,
 'avg10Volume': 87823778,
 'avg30Volume': 125804131,
 'day200MovingAvg': 157.4,
 'day50MovingAvg': 169.25,
 'employees': 152648,
 'ttmEPS': 11.45,
 'ttmDividendRate': 0.8981414346051175,
 'dividendYield': 0.005027877199535014,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-28',
 'nextEarningsDate': '2022-01-23',
 'peRatio': 15.730400860262797,
 'beta': 1.3855343486123621,
 'maxChangePercent': 68.05107831185238,
 'year5ChangePercent': 5.680203129605993,
 'year2ChangePercent': 1.5368094678017066,
 'year1ChangePercent': 0.3326519516771483,
 'ytdChangePercent': 0.37743729419552907,
 'month6ChangePercent': 0.34902326048474114,
 'month3ChangePercent': 0.2778140785938856,
 'month1ChangePercent': 0.14980418091739348,
 'day30ChangeP

In [4]:
data['year1ChangePercent']

0.3326519516771483

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_chunks = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_chunks)):
    symbol_strings.append(','.join(symbol_chunks[i]))
#    print(symbol_strings[i])

col = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
df = pd.DataFrame(columns=col)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    # ↑一次過攞曬symbol_string，唔好再逐個symbol for個api返黎
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index=col),ignore_index=True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,164.150,0.364267,N/A
1,AAL,18.490,0.161197,N/A
2,AAP,247.810,0.546357,N/A
3,AAPL,186.870,0.322436,N/A
4,ABBV,138.630,0.364407,N/A
...,...,...,...,...
500,YUM,139.650,0.278799,N/A
501,ZBH,134.920,-0.139224,N/A
502,ZBRA,620.750,0.568988,N/A
503,ZION,65.100,0.512445,N/A


In [7]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50]
df.reset_index(drop = True, inplace = True)# pandas要經常性 inplace = True
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.270,2.343141,N/A
1,DVN,44.554,2.107008,N/A
2,MCHP,90.770,1.67913,N/A
3,MRO,16.720,1.523372,N/A
4,FTNT,377.200,1.45497,N/A
5,FANG,114.552,1.418791,N/A
6,F,20.718,1.386491,N/A
7,NVDA,310.910,1.366368,N/A
8,NUE,118.930,1.263439,N/A
9,IT,347.130,1.12711,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    #OK, 如果輸錯點算？ try except
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(df.index)
for i in range(len(df)):
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df['Price'][i])
    # Number of Shares to Buy 取整數 捨小數點 math.floor
df

C:\Users\sitwa\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.270,2.343141,243
1,DVN,44.554,2.107008,448
2,MCHP,90.770,1.67913,220
3,MRO,16.720,1.523372,1196
4,FTNT,377.200,1.45497,53
5,FANG,114.552,1.418791,174
6,F,20.718,1.386491,965
7,NVDA,310.910,1.366368,64
8,NUE,118.930,1.263439,168
9,IT,347.130,1.12711,57


In [10]:
hqm_col = ['Ticker', 'Price', 'Number of Shares to Buy', 
               'One-Year Price Return', 
               'One-Year Return Percentile', 
               'Six-Month Price Return', 
               'Six-Month Return Percentile',
               'Three-Month Price Return',
               'Three-Month Return Percentile', 
               'One-Month Price Return',
               'One-Month Return Percentile',
               'HQM Score'
          ]
hqm_df = pd.DataFrame(columns= hqm_col)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    # ↑一次過攞曬symbol_string，唔好再逐個symbol for個api返黎
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],'N/A',
                                          data[symbol]['stats']['year1ChangePercent'],
                                          'N/A',
                                          data[symbol]['stats']['month6ChangePercent'],
                                          'N/A',
                                          data[symbol]['stats']['month3ChangePercent'],
                                          'N/A',
                                          data[symbol]['stats']['month1ChangePercent'],
                                          'N/A',
                                          'N/A'
                                         ],index=hqm_col),ignore_index=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.490,N/A,0.362128,N/A,0.079511,N/A,-0.003302,N/A,0.047748,N/A,N/A
1,AAL,18.370,N/A,0.159973,N/A,-0.135097,N/A,-0.14045,N/A,0.046137,N/A,N/A
2,AAP,241.240,N/A,0.535955,N/A,0.182167,N/A,0.114371,N/A,0.05688,N/A,N/A
3,AAPL,184.900,N/A,0.330203,N/A,0.349061,N/A,0.27638,N/A,0.146787,N/A,N/A
4,ABBV,137.060,N/A,0.363422,N/A,0.219548,N/A,0.270424,N/A,0.155858,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,143.390,N/A,0.288937,N/A,0.200333,N/A,0.133478,N/A,0.115438,N/A,N/A
501,ZBH,131.140,N/A,-0.140613,N/A,-0.217404,N/A,-0.152952,N/A,0.020697,N/A,N/A
502,ZBRA,599.880,N/A,0.585117,N/A,0.139469,N/A,0.127526,N/A,0.011564,N/A,N/A
503,ZION,65.600,N/A,0.515899,N/A,0.204116,N/A,0.012359,N/A,-0.016914,N/A,N/A


In [11]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']
hqm_df = hqm_df.fillna(value = np.nan) # ★任何計算都要numpy將N/A填充為np.nan可計算: .fillna(np.nan)

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100
        # percentileofscore(col,score)指成列數≤score的比例，如重複：搵平均值of(不包重複數比例+包重複數比例)/2
        

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.490,N/A,0.362128,0.607921,0.079511,0.489109,-0.003302,0.271287,0.047748,0.510891,N/A
1,AAL,18.370,N/A,0.159973,0.316832,-0.135097,0.085149,-0.140450,0.045545,0.046137,0.493069,N/A
2,AAP,241.240,N/A,0.535955,0.811881,0.182167,0.718812,0.114371,0.635644,0.056880,0.562376,N/A
3,AAPL,184.900,N/A,0.330203,0.568317,0.349061,0.938614,0.276380,0.934653,0.146787,0.944554,N/A
4,ABBV,137.060,N/A,0.363422,0.609901,0.219548,0.80198,0.270424,0.928713,0.155858,0.956436,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,143.390,N/A,0.288937,0.510891,0.200333,0.768317,0.133478,0.681188,0.115438,0.875248,N/A
501,ZBH,131.140,N/A,-0.140613,0.047525,-0.217404,0.029703,-0.152952,0.029703,0.020697,0.308911,N/A
502,ZBRA,599.880,N/A,0.585117,0.847525,0.139469,0.627723,0.127526,0.669307,0.011564,0.253465,N/A
503,ZION,65.600,N/A,0.515899,0.786139,0.204116,0.774257,0.012359,0.312871,-0.016914,0.126733,N/A


In [12]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row,f'{time_period} Return Percentile'])
    hqm_df.loc[row,'HQM Score'] = mean(momentum_percentiles)
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.490,N/A,0.362128,0.607921,0.079511,0.489109,-0.003302,0.271287,0.047748,0.510891,0.469802
1,AAL,18.370,N/A,0.159973,0.316832,-0.135097,0.085149,-0.140450,0.045545,0.046137,0.493069,0.235149
2,AAP,241.240,N/A,0.535955,0.811881,0.182167,0.718812,0.114371,0.635644,0.056880,0.562376,0.682178
3,AAPL,184.900,N/A,0.330203,0.568317,0.349061,0.938614,0.276380,0.934653,0.146787,0.944554,0.846535
4,ABBV,137.060,N/A,0.363422,0.609901,0.219548,0.80198,0.270424,0.928713,0.155858,0.956436,0.824257
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,143.390,N/A,0.288937,0.510891,0.200333,0.768317,0.133478,0.681188,0.115438,0.875248,0.708911
501,ZBH,131.140,N/A,-0.140613,0.047525,-0.217404,0.029703,-0.152952,0.029703,0.020697,0.308911,0.10396
502,ZBRA,599.880,N/A,0.585117,0.847525,0.139469,0.627723,0.127526,0.669307,0.011564,0.253465,0.599505
503,ZION,65.600,N/A,0.515899,0.786139,0.204116,0.774257,0.012359,0.312871,-0.016914,0.126733,0.5


In [16]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df.reset_index(drop = True, inplace = True)
hqm_df  = hqm_df[:50]
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,149.280,N/A,0.997029,0.968317,0.621056,0.984158,0.691515,0.992079,0.159727,0.960396,0.976238
1,CF,74.860,N/A,1.078943,0.972277,0.534324,0.980198,0.326916,0.960396,0.192098,0.982178,0.973762
2,AVGO,677.310,N/A,0.597548,0.861386,0.451075,0.972277,0.379205,0.978218,0.236231,0.988119,0.95
3,FTNT,376.890,N/A,1.522262,0.984158,0.498506,0.974257,0.283625,0.942574,0.120190,0.89505,0.94901
4,EXR,233.150,N/A,1.038098,0.970297,0.369013,0.946535,0.315462,0.956436,0.130907,0.914851,0.94703
5,ACN,420.030,N/A,0.629624,0.877228,0.440057,0.968317,0.287904,0.944554,0.175157,0.974257,0.941089
6,LYV,121.720,N/A,0.620709,0.873267,0.373909,0.950495,0.305951,0.950495,0.165374,0.970297,0.936139
7,PLD,166.960,N/A,0.743394,0.944554,0.376647,0.952475,0.326939,0.962376,0.115611,0.879208,0.934653
8,AZO,2105.068,N/A,0.729324,0.934653,0.389582,0.956436,0.226559,0.883168,0.138251,0.932673,0.926733
9,DRE,66.790,N/A,0.678864,0.916832,0.369603,0.948515,0.350963,0.966337,0.107213,0.845545,0.919307


In [17]:
portfolio_input()

Enter the size of your portfolio:1000000


In [22]:
position_size = float(portfolio_size)/len(hqm_df.index)
for row in hqm_df.index:
    hqm_df.loc[row,'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[row,'Price'])
hqm_df

C:\Users\sitwa\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,149.280,133,0.997029,0.968317,0.621056,0.984158,0.691515,0.992079,0.159727,0.960396,0.976238
1,CF,74.860,267,1.078943,0.972277,0.534324,0.980198,0.326916,0.960396,0.192098,0.982178,0.973762
2,AVGO,677.310,29,0.597548,0.861386,0.451075,0.972277,0.379205,0.978218,0.236231,0.988119,0.95
3,FTNT,376.890,53,1.522262,0.984158,0.498506,0.974257,0.283625,0.942574,0.120190,0.89505,0.94901
4,EXR,233.150,85,1.038098,0.970297,0.369013,0.946535,0.315462,0.956436,0.130907,0.914851,0.94703
5,ACN,420.030,47,0.629624,0.877228,0.440057,0.968317,0.287904,0.944554,0.175157,0.974257,0.941089
6,LYV,121.720,164,0.620709,0.873267,0.373909,0.950495,0.305951,0.950495,0.165374,0.970297,0.936139
7,PLD,166.960,119,0.743394,0.944554,0.376647,0.952475,0.326939,0.962376,0.115611,0.879208,0.934653
8,AZO,2105.068,9,0.729324,0.934653,0.389582,0.956436,0.226559,0.883168,0.138251,0.932673,0.926733
9,DRE,66.790,299,0.678864,0.916832,0.369603,0.948515,0.350963,0.966337,0.107213,0.845545,0.919307


In [24]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {'A': ['Ticker',string_template],
               'B': ['Price',dollar_template],
               'C': ['Number of Shares to Buy',integer_template] ,
               'D': ['One-Year Price Return', percent_template],
               'E': ['One-Year Return Percentile', percent_template],
               'F': ['Six-Month Price Return', percent_template],
               'G': ['Six-Month Return Percentile',percent_template],
               'H': ['Three-Month Price Return',percent_template],
               'I': ['Three-Month Return Percentile', percent_template],
               'J': ['One-Month Price Return',percent_template],
               'K': ['One-Month Return Percentile',percent_template],
               'L': ['HQM Score',percent_template]
              }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])#formating cells
    writer.sheets['Momentum Strategy'].write(f'{column}1', 25, column_formats[column][1])# formating headers

In [28]:
writer.save() #記住啊